In [2]:
import pandas as pd 
import numpy as np
import os
import shutil
import re
import math 
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("expand.frame_repr",False)
print("import succed!")

import succed!


In [ ]:
# 设置——两库全量_and_疾病标签_总表的导入路径
input_path = r'/Users/edz/Documents/yandi_data/AI_data/zn_dc_ai总表'
input_path_file = r'/两库全量_and_疾病标签_总表.xlsx'

In [ ]:
df = pd.read_excel(input_path+input_path_file,sep='\t',index_col=None)
df.tail(2)

In [ ]:
df.shape  # 查看行列容量

In [ ]:
# # 筛选有效列数据
# df_pre = df[['来自图片id','来自检查id','阅片诊断','图像地址','sBosobject','图像md5信息','sImageMd5','arithmeticTag','nTagCheckDevice',
#             'left_糖网','right_糖网','left_青光眼','right_青光眼','left_屈光不正','right_屈光不正','left_白内障','right_白内障']]

In [ ]:
df_pre.info()

## 合并剩余37万数据的32分类标签

In [ ]:
# 设置读入32分类标签的数据路径
input_path_32 = r'/Users/edz/Documents/yandi_data/AI_data/32分类标签/32_hou_label' 
dir_list_32 = os.listdir(input_path_32)
file_dir_list_32 = [ os.path.join(input_path_32,x) for  x in dir_list_32 ]
print(file_dir_list_32)

In [ ]:
df_classific_32 = pd.DataFrame()
# 循环读入32分类标签数据
for i in range(1,len(file_dir_list_32)):
    df_i = pd.read_csv(file_dir_list_32[i],sep='\s+',index_col=None,header=None)
    print("{}的数据容量为{}：".format(file_dir_list_32[i], df_i.shape[0]))
    df_classific_32 = df_classific_32.append(df_i)
# 重命名列名
df_classific_32.rename(columns={0:'图像地址',1:'md5值',2:'arithmeticTag_32'},inplace=True)
print(df_classific_32.shape[0])

In [ ]:
df_classific_32.reset_index(drop=True,inplace=True)  # 重置索引
df_classific_32.tail(2)

In [ ]:
df_classific_32['arithmeticTag_32'].notnull().sum()  # 查看无32分类标签的数据容量

In [ ]:
df = df.merge(df_classific_32,how='left', on='图像地址')

In [ ]:
df.info()

### pd.isnull(z.at[815076,'arithmeticTag'])==True  # Dataframe判断一列中某值是否为空

In [ ]:
# 循环用有效的'arithmeticTag_32'值，代替'arithmeticTag'中的空值
for j in range(df.shape[0]):
    if pd.isnull(df.at[j,'arithmeticTag'])==True:   # Dataframe判断一列中某值是否为空
        df.at[j,'arithmeticTag'] = df.at[j,'arithmeticTag_32']  # 填充指定值
    else:
        pass

In [ ]:
df['arithmeticTag'].notnull().sum()

In [ ]:
df['arithmeticTag'].value_counts()

In [ ]:
df.drop(columns=['arithmeticTag_32'],inplace=True)

#### 根据有"nTagCheckDevice"字段的根据字段内容，定义图像类型；没有的则根据"arithmeticTag"(即：32分类)字段内容，定义图像类型。

In [ ]:
# 32分类标签的辅助修正
# 插入”图像类别“标签列
insert_class_num = df.columns.get_loc('arithmeticTag')
print(insert_class_num)

In [ ]:
df['nTagCheckDevice'].value_counts()

In [ ]:
df.insert(insert_class_num+1,'图像类别',None)  # 获取插入列的索引

In [ ]:
df['图像类别'] = df['arithmeticTag']

In [ ]:
df.columns

In [ ]:
for i in range(df.shape[0]):
    if df.at[i,'nTagCheckDevice']=='眼底照相机':
        df.at[i,'图像类别'] = 11
    elif df.at[i,'nTagCheckDevice']=='OCT':
        df.at[i,'图像类别'] = 25
    else:
        pass

In [ ]:
# 读入左右眼标签列
input_path_lr = r'/Users/edz/Documents/yandi_data/AI_data/左右眼label'
select_filelist = os.listdir(input_path_lr)  # 获取被筛选文件夹中所有的文件名称
file_dir_list = [os.path.join(input_path_lr,x) for x in select_filelist]
print(file_dir_list)  # 绝对路径

In [ ]:
df_lr = pd.DataFrame()  # 用于存储读入表
# 循环读入文件
# 该文本中的分割符既有空格又有制表符（‘/t’），sep参数用‘/s+’，可以匹配任何空格。
for i in range(1, len(file_dir_list)):
    df_i = pd.read_csv( file_dir_list[i], sep='\s+',index_col=None,header=None)
    print("读入{},shape:{}".format( file_dir_list[i], df_i.shape[0]))
    df_lr = df_lr.append(df_i,ignore_index=True)
print("合表后，左右眼标签表的数据容量{}".format(df_lr.shape[0]))

In [ ]:
df_lr.rename(columns={0:'图像地址',1:'MD5值',2:'左右眼'},inplace=True)
df_lr.tail(2)

In [ ]:
df_lr['左右眼'].value_counts()

In [ ]:
df_lr['左右眼'].isnull().sum()

In [ ]:
df_lr[df_lr['左右眼'].isnull()==True].head(2)

In [ ]:
df_lr.tail(2)  

pandas中规定，只要数值类型列，有缺失值，那么这一列的数据类型就是浮点型，所以不能强制转换为整型，不然会报错——ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df = df.merge(df_lr,how='left',on='图像地址')

In [ ]:
df.info()

In [ ]:
pd.isnull(df.loc[0,'MD5值'])==False

In [ ]:
a = df[df['图像md5信息'].duplicated()==True]

In [ ]:
a['图像md5信息'].value_counts()

In [ ]:
df['图像md5信息'].str.contains('empty').sum()

In [ ]:
# 整合MD5值的相关信息
for i in range(df.shape[0]):
    if df.loc[i,'图像md5信息'] =='empty':
        if pd.isnull(df.loc[i,'MD5值'])==False:
            df.loc[i,'图像md5信息'] = df.loc[i,'MD5值']
        else:
            pass
    elif df.loc[i,'图像md5信息'] =='no':
        if pd.isnull(df.loc[i,'MD5值'])==False:
            df.loc[i,'图像md5信息'] = df.loc[i,'MD5值']
        else:
            pass
    else:
        pass

In [ ]:
# 整合MD5值的相关信息
for i in range(df.shape[0]):
    if df.loc[i,'图像md5信息'] =='empty':
        if pd.isnull(df.loc[i,'md5值'])==False:
            df.loc[i,'图像md5信息'] = df.loc[i,'md5值']
        else:
            pass
    elif df.loc[i,'图像md5信息'] =='no':
        if pd.isnull(df.loc[i,'md5值'])==False:
            df.loc[i,'图像md5信息'] = df.loc[i,'md5值']
        else:
            pass
    else:
        pass

In [ ]:
df_pre.columns

In [ ]:
# 筛选有效列数据
df_pre = df[['来自图片id','图像地址','图像md5信息','左右眼','图像类别','影像描述','阅片诊断','来自检查id','nTagCheckDevice',
            'left_糖网','right_糖网','left_青光眼','right_青光眼','left_屈光不正','right_屈光不正','left_白内障','right_白内障']]

In [ ]:
df_zong['图像md5信息'].str.contains('empty').sum()

### 读入疾病标签表

In [ ]:
input_path_bq = r'/Users/edz/Documents/yandi_data/AI_data/zn_ai_疾病标签'
dir_list_bq = os.listdir(input_path_bq)
print(dir_list_bq)

In [ ]:
# 删除指定的无需读入列
del_list = ['.DS_Store','糖网_quan_250625.xlsx','白内障_quan_250625.xlsx','青光眼_quan_250625.xlsx','屈光不正_quan_250625.xlsx']
for i in del_list:
    dir_list_bq.remove(i)  # python中删除list中某指定元素,知道索引的话就可以用del来删除 del list[i]

In [ ]:
file_dir_list_bq = [os.path.join(input_path_bq, x) for x in dir_list_bq]
print(file_dir_list_bq) 

In [ ]:
# 读入标签数据
for i in range(len(file_dir_list_bq)):
    df_bq_i = pd.read_excel(file_dir_list_bq[i], usecols=[0,2,3], sep='\t',index_col=None) 
    if i == 0:
        df_bq = df_bq_i  # 首次赋值==原始表
    else:
        df_bq = df_bq.merge(df_bq_i,how='left',on='来自检查id')  # 之后添加新标签表

print(df_bq.columns)
    # df_bq_1.head(2)

In [ ]:
df_zong = df_pre.merge(df_bq, how='left', on='来自检查id')

In [ ]:
df_zong.head(2)

In [ ]:
df_zong['图像类别'].notnull().sum()

In [ ]:
df_zong.shape[0]

### 导出合并右右眼标签、32分类标签、疾病标签的总数据表

In [ ]:
df_zong.columns

In [ ]:
df_zong['图像类别'].notnull().sum()  # 存在32分类标签图片的数据总量

In [ ]:
df_zong['图像类别'].isnull().sum()  # 不存在32分类标签图片的数据总量

### 筛选眼底图数据——用于统计表的制作
###### 取差集 c = list(set(a).difference(set(b)))

In [ ]:
a = df_zong.columns.tolist()

In [ ]:
# 获取合并标签列list
a = df_zong.columns.tolist()
a =  [a[i].replace('right_','') for i in range(len(a))]  # replace函数实现str的替换
a =  [a[i].replace('left_','') for i in range(len(a))]
start_index = df_zong.columns.get_loc('nTagCheckDevice')
b = a[start_index+1:]
# 得到新增列名称的列表
x = list(set(b))  # 去重得到新增项列表名称——合并标签
print(x)  #  新增列名称的列表

In [ ]:
insert_index = df_zong.columns.get_loc('阅片诊断')  # 获取“阅片诊断”列的index号
insert_columns_list = x  # 新增列名称的列表
# 插入指定列
for i in range(len(insert_columns_list)):
    df_zong.insert(insert_index+(i+1),insert_columns_list[i],None)

In [ ]:
print(df_zong.columns)

In [ ]:
# 根据左右眼标签识别唯一的眼底图对应疾病标签
for k in range(len(x)):
    for i in range(df_zong.shape[0]):  
        str_i = df_zong.at[i,'左右眼']
        if  str_i == 0: # 左眼
            df_zong.at[i, x[k]] = df_zong.at[i,'left_' + x[k]]
        elif str_i == 1:
            df_zong.at[i, x[k]] = df_zong.at[i,'right_' + x[k]]
        else:
            pass

In [ ]:
df_zong.loc[20:25,:]  # 查验

In [ ]:
# 根据”左眼标签==右眼标签“识别图片对应的疾病标签
def lr_equal_func(df,l_column,r_column,equal_column):
    for j in range(df.shape[0]):
        if pd.isnull(df.at[j,equal_column])==False:
            pass
        else:
            if df.at[j,l_column] == df.at[j,r_column]:
                df.at[j,equal_column] = df.at[j,l_column]
    #             print(df_last.at[j,'糖网'])
            else:
                pass    

In [ ]:
# # 调用疾病标签合并的打标函数
for k in range(len(x)):
    print("处理——{}列：".format(x[k]))
    lr_equal_func(df_zong,'left_' + x[k],'right_' + x[k], x[k])

In [ ]:
df_zong.info()  # 校验已打标数据总量

In [ ]:
df_zong.columns

## 导出全量疾病标签数据

In [ ]:
# 导出最终的全量总表——右、右眼两列疾病标签
out_put_path = r'/Users/edz/Documents/yandi_data/AI_data/zn_dc_ai总表/影像描述_异常病灶总表'
out_put_file = r'/阅片诊断_疾病标签_59类_总表_815098.xlsx'
writer = pd.ExcelWriter(out_put_path + out_put_file)
df_zong.to_excel(writer,sheet_name='607971条已解析图像类别',index=None)
writer.save()

#### 筛选眼底图像数据

In [ ]:
# 筛选眼底图像数据
df_zong_yd = df_zong[df_zong['图像类别']==11]
df_zong_yd.reset_index(drop=True,inplace=True)

In [ ]:
df_zong_yd.tail(2)

In [ ]:
df_zong_yd.shape[0]

## 导出眼底图疾病标签数据

In [ ]:
out_put_path_yd_ydb = r'/Users/edz/Documents/yandi_data/AI_data/zn_dc_ai总表/影像描述_异常病灶总表'
out_put_file_yd_ydb = r'/眼底图_阅片诊断_疾病标签_59类_总表_504334.xlsx'
writer = pd.ExcelWriter(out_put_path_yd_ydb + out_put_file_yd_ydb)
df_zong_yd.to_excel(writer,sheet_name='已打标的眼底图标签表_'+ str(df_zong_yd.shape[0]),index=None)
writer.save()

## 导出最终的全量数据表

In [5]:
# 读入疾病信息表
in_put_path_bz = r'/Users/edz/Documents/yandi_data/AI_data/zn_dc_ai总表/影像描述_异常病灶总表'
in_put_file_bz = r'/阅片诊断_疾病标签_59类_总表_815098.xlsx'
df_jibing = pd.read_excel(in_put_path_bz+in_put_file_bz,sep='\t',index_col=None)

In [6]:
df_jb = df_jibing

In [7]:
a_list = df_jb.columns
print(a_list)

Index(['来自图片id', '图像地址', '图像md5信息', '左右眼', '图像类别', '影像描述', '阅片诊断', '视神经炎',
       '眼底血管样条纹', 'Purtscher视网膜病变',
       ...
       'left_脉络膜萎缩', 'right_脉络膜萎缩', 'left_脉络膜病变', 'right_脉络膜病变', 'left_人工晶体眼',
       'right_人工晶体眼', 'left_玻璃体后脱离', 'right_玻璃体后脱离', 'left_斜视', 'right_斜视'],
      dtype='object', length=186)


In [8]:
df_jb = df_jb.drop(columns = ['图像地址', '图像md5信息', '左右眼', '图像类别', 
                              '影像描述','阅片诊断','来自检查id','nTagCheckDevice'])

In [3]:
in_put_path_bz = r'/Users/edz/Documents/yandi_data/AI_data/zn_dc_ai总表/影像描述_异常病灶总表'
in_put_file_bz = r'/影像描述_异常病灶标签_40类_总表_815098.xlsx'
df_bz = pd.read_excel(in_put_path_bz+in_put_file_bz,sep='\t',index_col=None)

In [9]:
df_bingzao = df_bz

In [10]:
df_bingzao = df_bingzao.drop(columns = ['图像地址', '图像md5信息', '左右眼', '图像类别', 
                              '影像描述','阅片诊断','来自检查id','nTagCheckDevice'])

In [4]:
# 读入基本信息表
in_put_path_bz = r'/Users/edz/Documents/yandi_data/AI_data/zn_dc_ai总表/影像描述_异常病灶总表'
in_put_file_bz = r'/阅片诊断非空_基本信息_总表_29列_815098.xlsx'
df_jiben = pd.read_excel(in_put_path_bz+in_put_file_bz,sep='\t',index_col=None)

In [ ]:
df_jiben.columns

In [ ]:
df_jiben.columns.get_loc('影像描述')

In [11]:
# 合并疾病标签和异常病灶标签
df_add = df_bingzao.merge(df_jb,how='left',on='来自图片id')

In [ ]:
df_he = df_jiben.merge(df_add, how='left', on='来自图片id')

In [ ]:
add_list = df_he.columns.to_list()
add_list

In [ ]:
print(add_list)

### 设置变量，统计各疾病标签下的数据量

In [ ]:
##################第一步，统计已打标的图像数量
## 以视网膜血管阻塞为例设计统计函数
def son_statistic_func(df,column_name,contain_strs):
    a = df[df[column_name].notnull()]
    b = a[a[column_name].str.contains(contain_strs)]
    # print(b['视网膜血管阻塞'].value_counts())
    # print(b.shape[0])
    c = pd.DataFrame(data=b[column_name].value_counts())
    c.reset_index(level=0,inplace=True)  # 将索引列转为普通列
    c.rename(columns={c.columns[0]:'疾病子类',c.columns[1]:'已打标眼底图数量'},inplace=True)
    # 新增子类合计行，显示为标准表
    c.loc[c.shape[0]]=c.loc[:,['已打标眼底图数量']].apply(lambda x: x.sum())  # 新增一行用于统计指定列下，各行数据总和，并作为新行添加到末尾
    c.loc[c.shape[0]-1,'疾病子类'] = column_name + '类合计'
    c['已打标眼底图数量'] = c['已打标眼底图数量'].astype(int)  # 将数据强制转为整型
#     print(c)
    return c

#### 糖网、白内障、青光眼

In [ ]:
ydb_table = pd.DataFrame()

In [ ]:
########## 糖网
hb_part_tw = son_statistic_func(df_last_yd,'糖网','糖网')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '糖网'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = hb_part_tw.loc[hb_part_tw.shape[0]-1,'已打标眼底图数量']

In [ ]:
########## 白内障
bnz_part = son_statistic_func(df_last_yd,'白内障','白内障')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '白内障'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = bnz_part.loc[bnz_part.shape[0]-1,'已打标眼底图数量']

In [ ]:
########## 青光眼
qgy_part = son_statistic_func(df_last_yd,'青光眼','青光眼')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '青光眼'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = qgy_part.loc[qgy_part.shape[0]-1,'已打标眼底图数量']

In [ ]:
ydb_table

#### 视网膜血管阻塞

In [ ]:
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜血管阻塞','视网膜血管阻塞'),ignore_index=True)

#### 视网膜血管炎

In [ ]:
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜血管炎','视网膜血管炎'),ignore_index=True)

#### 黄斑病变

In [ ]:
########## AMD
hb_part_amd = son_statistic_func(df_last_yd,'黄斑病变','AMD')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '老年性黄斑变性（AMD）'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = hb_part_amd.loc[hb_part_amd.shape[0]-1,'已打标眼底图数量']
########## 黄斑前膜
hb_part_qm = son_statistic_func(df_last_yd,'黄斑病变','黄斑前膜')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '黄斑前膜'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = hb_part_qm.loc[hb_part_qm.shape[0]-1,'已打标眼底图数量']
########## 黄斑裂孔
hb_part_lk = son_statistic_func(df_last_yd,'黄斑病变','黄斑裂孔')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '黄斑裂孔'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = hb_part_lk.loc[hb_part_lk.shape[0]-1,'已打标眼底图数量']
########## 黄斑区色素上皮
hb_part_sssp = son_statistic_func(df_last_yd,'黄斑病变','黄斑区色素上皮')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '黄斑区色素上皮改变、脱离'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = hb_part_sssp.loc[hb_part_sssp.shape[0]-1,'已打标眼底图数量']
########## 黄斑病变,瘢痕期|黄斑病变,早期
hb_part_hbb = son_statistic_func(df_last_yd,'黄斑病变','黄斑病变,瘢痕期|黄斑病变,早期')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '黄斑病变,瘢痕期|黄斑病变,早期'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = hb_part_hbb.loc[hb_part_hbb.shape[0]-1,'已打标眼底图数量']
# 黄斑病变子标签添加
hb_part = son_statistic_func(df_last_yd,'黄斑病变','黄斑病变')
ydb_table = ydb_table.append(hb_part[~hb_part['疾病子类'].str.contains('AMD|黄斑区色素上皮|黄斑裂孔|黄斑前膜|瘢痕期|早期')],ignore_index=True)

In [ ]:
ydb_table

In [ ]:
# 合并行函数
def hebing_hang_func(df_table,delete_hang,mubiao_hang):
    # 寻找合并子行的索引
    b = df_table[df_table['疾病子类'].isin([delete_hang])].index.tolist()
    # 寻找合并目标行的索引
    a = df_table[df_table['疾病子类'].isin([mubiao_hang])].index.tolist()
    df_table.at[a[0],'已打标眼底图数量'] += df_table.at[b[0],'已打标眼底图数量']
    df_table = df_table.drop(b[0],axis=0)  # 删除指定index的行数据
    return df_table

In [ ]:
# 调用合并行函数，合并部分行数据
ydb_table = hebing_hang_func(ydb_table,'黄斑病变,黄斑水肿,糖尿病性','黄斑病变,黄斑水肿')
ydb_table = hebing_hang_func(ydb_table,'黄斑病变,瘢痕期|黄斑病变,早期','黄斑病变')
ydb_table = hebing_hang_func(ydb_table,'黄斑病变,营养不良,先天性','黄斑病变,营养不良')

# 修改部分行疾病的名称为指定内容
c = ydb_table[ydb_table['疾病子类'].isin(['黄斑病变,黄斑水肿,黄斑囊样水肿'])].index.tolist() 
ydb_table.at[c[0],'疾病子类'] = '黄斑囊样水肿'
d = ydb_table[ydb_table['疾病子类'].isin(['黄斑病变,黄斑囊样变性'])].index.tolist() 
ydb_table.at[d[0],'疾病子类'] = '黄斑囊样变性'
e = ydb_table[ydb_table['疾病子类'].isin(['黄斑病变,黄斑变性,先天性,Stargardt病'])].index.tolist() 
ydb_table.at[e[0],'疾病子类'] = '先天性黄斑变性Stargardt病'
f = ydb_table[ydb_table['疾病子类'].isin(['黄斑病变,黄斑变性,先天性,Best病'])].index.tolist() 
ydb_table.at[f[0],'疾病子类'] = '先天性黄斑变性Best病'

In [ ]:
ydb_table

In [ ]:
ydb_table.reset_index(drop=True,inplace=True)   # 重置index
# 获取删除行的起始index和终止index
hb_delete_start =  ydb_table[ydb_table['疾病子类'].isin(['老年性黄斑变性（AMD）'])].index.tolist()
hb_delete_end = ydb_table[ydb_table['疾病子类'].isin(['黄斑病变类合计'])].index.tolist()
# 筛选指定行内容并排序
hbbb_sort = ydb_table[hb_delete_start[0]:hb_delete_end[0]].sort_values('已打标眼底图数量',ascending=False) # 筛选排序行内容 
ydb_table = ydb_table.drop([i for i in range(hb_delete_start[0],hb_delete_end[0]+1)],axis=0)   # 根据行列表一次性删除index的多行
ydb_table = ydb_table.append(hbbb_sort,ignore_index=True)

In [ ]:
########## 黄斑病变——合计行添加
qgy_part = son_statistic_func(df_last_yd,'黄斑病变','黄斑病变')
# 指定行新数据
ydb_table.loc[ydb_table.shape[0],'疾病子类'] = '黄斑病变类合计'
ydb_table.loc[ydb_table.shape[0]-1,'已打标眼底图数量'] = qgy_part.loc[qgy_part.shape[0]-1,'已打标眼底图数量']

In [ ]:
ydb_table

#### 屈光不正

In [ ]:
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'屈光不正','屈光不正'),ignore_index=True)

#### 视网膜有髓神经纤维

In [ ]:
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜有髓神经纤维','视网膜有髓神经纤维'),ignore_index=True)

#### 眼底动脉硬化

In [ ]:
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'眼底动脉硬化','眼底动脉硬化'),ignore_index=True)

#### 视网膜脱离  视网膜色素上皮改变 脉络膜血管瘤 脉络膜缺损 黑色素瘤 色素痣 脉络膜病变 玻璃体浑浊 玻璃体后脱离 视盘水肿 玻璃体出血 大视杯 视盘缺损 视盘发育异常 视神经炎

In [ ]:
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜脱离','视网膜脱离'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜色素上皮改变','视网膜色素上皮改变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'脉络膜萎缩','脉络膜萎缩'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'脉络膜血管瘤','脉络膜血管瘤'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'脉络膜缺损','脉络膜缺损'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'黑色素瘤','黑色素瘤'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'色素痣','色素痣'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'脉络膜病变','脉络膜病变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'玻璃体浑浊','玻璃体浑浊'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'玻璃体后脱离','玻璃体后脱离'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视盘水肿','视盘水肿'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'玻璃体出血','玻璃体出血'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'大视杯','大视杯'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视盘缺损','视盘缺损'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视盘发育异常','视盘发育异常'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视神经炎','视神经炎'),ignore_index=True)

#### '视网膜色素变性', '视网膜震荡', '视神经萎缩', 'Coats病', '早产儿视网膜病变ROP', '占位性病变','视网膜劈裂症', '缺血性视神经病变', '屈光间质浑浊', '结膜炎', 'Purtscher视网膜病变','缺血性视网膜病变', '息肉样脉络膜血管病变PCV','色素膜炎', '后发障', '斜视', '人工晶体眼','脉络膜骨瘤', '特发性脉络膜新生血管', '视网膜前膜', '结晶样视网膜变性', '视神经病变','病理性近视', '星状玻璃体变性', '晶体脱位', '高血压性视网膜病变', '巩膜炎', '闪辉性玻璃体液化',  '眼底出血', '虹膜炎', '无晶状体眼', '视网膜大动脉瘤', '眼底血管样条纹', '黑色素细胞瘤'

In [ ]:
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜色素变性','视网膜色素变性'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜震荡','视网膜震荡'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视神经萎缩','视神经萎缩'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'Coats病','Coats病'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'早产儿视网膜病变ROP','ROP'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'占位性病变','占位性病变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜劈裂症','视网膜劈裂症'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'缺血性视神经病变','缺血性视神经病变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'屈光间质浑浊','屈光间质浑浊'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'结膜炎','结膜炎'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'Purtscher视网膜病变','Purtscher视网膜病变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'缺血性视网膜病变','缺血性视网膜病变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'息肉样脉络膜血管病变PCV','息肉样脉络膜血管病变PCV'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'色素膜炎','色素膜炎'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'斜视','斜视'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'人工晶体眼','人工晶体眼'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'脉络膜骨瘤','脉络膜骨瘤'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'特发性脉络膜新生血管','特发性脉络膜新生血管'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜前膜','视网膜前膜'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'结晶样视网膜变性','结晶样视网膜变性'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视神经病变','视神经病变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'病理性近视','病理性近视'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'星状玻璃体变性','星状玻璃体变性'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'晶体脱位','晶体脱位'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'高血压性视网膜病变','高血压性视网膜病变'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'巩膜炎','巩膜炎'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'闪辉性玻璃体液化','闪辉性玻璃体液化'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'眼底出血','眼底出血'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'虹膜炎','虹膜炎'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'无晶状体眼','无晶状体眼'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'视网膜大动脉瘤','视网膜大动脉瘤'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'眼底血管样条纹','眼底血管样条纹'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'黑色素细胞瘤','黑色素细胞瘤'),ignore_index=True)
ydb_table = ydb_table.append(son_statistic_func(df_last_yd,'后发障','后发障'),ignore_index=True)

In [ ]:
ydb_table

### 筛选——可打标眼底图像数量统计
筛选条件：”图像类别“==眼底图，缺少左、右眼标签，而且左、右眼疾病种类不同

In [ ]:
# 判断左右糖网是否均为空
# pd.isnull(df_last_yd.at[0,'left_糖网']) and pd.isnull(df_last_yd.at[0,'right_糖网'])

In [ ]:
rop

In [ ]:
# 逐行添加”未打标的含指定疾病的“，无左右眼标签，且左右眼疾病标签不同的行数据
def not_label_func(df,l_column,r_column):
    df_lr_dif = pd.DataFrame()
    for i in range(df.shape[0]):
        if pd.isnull(df.at[i,l_column]) and pd.isnull(df.at[i,r_column]):
            pass
        else:
            if df.at[i,l_column] == df.at[i,r_column]:
                pass
            else:
                df_lr_dif = df_lr_dif.append(pd.DataFrame(data=df.iloc[i:i+1,:]))
    print("左右眼疾病标签不一致的shape：{}".format(df_lr_dif.shape[0]))
    df_lr_weizhi = df_lr_dif[df_lr_dif['左右眼']==2]
    df_lr_isnull = df_lr_dif[df_lr_dif['左右眼'].isnull()]
    print("左右眼疾病标签不一致的标签数据中——左右眼==2未知的shape：{}".format(df_lr_weizhi.shape[0]))
    print("左右眼疾病标签不一致的标签数据中——左右眼isnull的shape：{}".format(df_lr_isnull.shape[0]))
    df_wdb = df_lr_weizhi.append(df_lr_isnull,ignore_index=True)
    print( l_column[5:]+"无左右眼，左右疾病不一致的眼底图shape：{}".format(df_wdb.shape[0]))
    return df_wdb

In [ ]:
df_wdb_last_tw = not_label_func(df_last_yd,'left_糖网','right_糖网')
df_wdb_last_qgy = not_label_func(df_last_yd,'left_青光眼','right_青光眼')
df_wdb_last_bnz = not_label_func(df_last_yd,'left_白内障','right_白内障')
df_wdb_last_qgbz = not_label_func(df_last_yd,'left_屈光不正','right_屈光不正')

In [ ]:
df_wdb_last_hbbb = not_label_func(df_last_yd,'left_黄斑病变','right_黄斑病变')

In [ ]:
df_wdb_last_xgzs = not_label_func(df_last_yd,'left_视网膜血管阻塞','right_视网膜血管阻塞')
df_wdb_last_xgy = not_label_func(df_last_yd,'left_视网膜血管炎','right_视网膜血管炎')
df_wdb_last_ys = not_label_func(df_last_yd,'left_视网膜有髓神经纤维','right_视网膜有髓神经纤维')
df_wdb_last_dmyh = not_label_func(df_last_yd,'left_眼底动脉硬化','right_眼底动脉硬化')
df_wdb_last_swtl = not_label_func(df_last_yd,'left_视网膜脱离','right_视网膜脱离')
df_wdb_last_ssspgb = not_label_func(df_last_yd,'left_视网膜色素上皮改变','right_视网膜色素上皮改变')
df_wdb_last_mlmws = not_label_func(df_last_yd,'left_脉络膜萎缩','right_脉络膜萎缩')
df_wdb_last_mlmwsmxgl = not_label_func(df_last_yd,'left_脉络膜血管瘤','right_脉络膜血管瘤')

In [ ]:
df_wdb_last_mlmqs = not_label_func(df_last_yd,'left_脉络膜缺损','right_脉络膜缺损')
df_wdb_last_hssl = not_label_func(df_last_yd,'left_黑色素瘤','right_黑色素瘤')
df_wdb_last_ssz = not_label_func(df_last_yd,'left_色素痣','right_色素痣')
df_wdb_last_mlmbb = not_label_func(df_last_yd,'left_脉络膜病变','right_脉络膜病变')
df_wdb_last_blthz = not_label_func(df_last_yd,'left_玻璃体浑浊','right_玻璃体浑浊')
df_wdb_last_blthzthtl = not_label_func(df_last_yd,'left_玻璃体后脱离','right_玻璃体后脱离')
df_wdb_last_spsz = not_label_func(df_last_yd,'left_视盘水肿','right_视盘水肿')
df_wdb_last_bltcx = not_label_func(df_last_yd,'left_玻璃体出血','right_玻璃体出血')
df_wdb_last_dsb = not_label_func(df_last_yd,'left_大视杯','right_大视杯')
df_wdb_last_spszqs = not_label_func(df_last_yd,'left_视盘缺损','right_视盘缺损')
df_wdb_last_spfyyc = not_label_func(df_last_yd,'left_视盘发育异常','right_视盘发育异常')
df_wdb_last_ssjy = not_label_func(df_last_yd,'left_视神经炎','right_视神经炎')

In [ ]:
# 第二批
df_wdb_last_01 = not_label_func(df_last_yd,'left_视网膜色素变性','right_视网膜色素变性')
df_wdb_last_02 = not_label_func(df_last_yd,'left_视网膜震荡','right_视网膜震荡')
df_wdb_last_03 = not_label_func(df_last_yd,'left_视神经萎缩','right_视神经萎缩')
df_wdb_last_04 = not_label_func(df_last_yd,'left_Coats病','right_Coats病')
# df_wdb_last_05 = not_label_func(df_last_yd,'left_早产儿视网膜病变ROP','right_早产儿视网膜病变ROP')   # 不存在未打标的数据，故未打标DataFrame为空，统计报错
df_wdb_last_06 = not_label_func(df_last_yd,'left_占位性病变','right_占位性病变')
df_wdb_last_07 = not_label_func(df_last_yd,'left_视网膜劈裂症','right_视网膜劈裂症')
df_wdb_last_08 = not_label_func(df_last_yd,'left_缺血性视神经病变','right_缺血性视神经病变')
df_wdb_last_09 = not_label_func(df_last_yd,'left_屈光间质浑浊','right_屈光间质浑浊')

In [ ]:
df_wdb_last_10 = not_label_func(df_last_yd,'left_结膜炎','right_结膜炎')
df_wdb_last_11 = not_label_func(df_last_yd,'left_Purtscher视网膜病变','right_Purtscher视网膜病变')
df_wdb_last_12 = not_label_func(df_last_yd,'left_缺血性视网膜病变','right_缺血性视网膜病变')
df_wdb_last_13 = not_label_func(df_last_yd,'left_息肉样脉络膜血管病变PCV','right_息肉样脉络膜血管病变PCV')
df_wdb_last_14 = not_label_func(df_last_yd,'left_色素膜炎','right_色素膜炎')
df_wdb_last_15 = not_label_func(df_last_yd,'left_后发障','right_后发障')
df_wdb_last_16 = not_label_func(df_last_yd,'left_斜视','right_斜视')
df_wdb_last_17 = not_label_func(df_last_yd,'left_人工晶体眼','right_人工晶体眼')
df_wdb_last_18 = not_label_func(df_last_yd,'left_脉络膜骨瘤','right_脉络膜骨瘤')

In [ ]:
df_wdb_last_19 = not_label_func(df_last_yd,'left_特发性脉络膜新生血管','right_特发性脉络膜新生血管')
df_wdb_last_20 = not_label_func(df_last_yd,'left_视网膜前膜','right_视网膜前膜')
# df_wdb_last_21 = not_label_func(df_last_yd,'left_结晶样视网膜变性','right_结晶样视网膜变性')
df_wdb_last_22 = not_label_func(df_last_yd,'left_视神经病变','right_视神经病变')
df_wdb_last_23 = not_label_func(df_last_yd,'left_病理性近视','right_病理性近视')
df_wdb_last_24 = not_label_func(df_last_yd,'left_星状玻璃体变性','right_星状玻璃体变性')
df_wdb_last_25 = not_label_func(df_last_yd,'left_晶体脱位','right_晶体脱位')
df_wdb_last_26 = not_label_func(df_last_yd,'left_高血压性视网膜病变','right_高血压性视网膜病变')
df_wdb_last_27 = not_label_func(df_last_yd,'left_巩膜炎','right_巩膜炎')

In [ ]:
df_wdb_last_28 = not_label_func(df_last_yd,'left_闪辉性玻璃体液化','right_闪辉性玻璃体液化')
df_wdb_last_29 = not_label_func(df_last_yd,'left_眼底出血','right_眼底出血')
df_wdb_last_30 = not_label_func(df_last_yd,'left_虹膜炎','right_虹膜炎')
df_wdb_last_31 = not_label_func(df_last_yd,'left_无晶状体眼','right_无晶状体眼')
df_wdb_last_32 = not_label_func(df_last_yd,'left_视网膜大动脉瘤','right_视网膜大动脉瘤')
df_wdb_last_33 = not_label_func(df_last_yd,'left_眼底血管样条纹','right_眼底血管样条纹')
df_wdb_last_34 = not_label_func(df_last_yd,'left_黑色素细胞瘤','right_黑色素细胞瘤')

In [ ]:
rop

In [ ]:
###### 绘制未打标标签表
wdb_table = pd.DataFrame(columns=['疾病子类','未打标眼底图数量']) 

In [ ]:
# '早产儿视网膜病变ROP类合计','结晶样视网膜变性类合计',

In [ ]:
wdb_table['疾病子类'] = ['糖网','白内障','青光眼','屈光不正类合计','黄斑病变类合计','视网膜血管阻塞类合计','视网膜血管炎类合计',
                     '视网膜有髓神经纤维类合计','眼底动脉硬化类合计','视网膜脱离类合计','视网膜色素上皮改变类合计','脉络膜萎缩类合计',
                     '脉络膜血管瘤类合计','脉络膜缺损类合计','黑色素瘤类合计','色素痣类合计','脉络膜病变类合计','玻璃体浑浊类合计',
                     '玻璃体后脱离类合计','视盘水肿类合计','玻璃体出血类合计','大视杯类合计','视盘缺损类合计','视盘发育异常类合计',
                     '视神经炎类合计','视网膜色素变性类合计', '视网膜震荡类合计', '视神经萎缩类合计', 'Coats病类合计', 
                     '占位性病变类合计','视网膜劈裂症类合计', '缺血性视神经病变类合计', '屈光间质浑浊类合计', '结膜炎类合计', 
                     'Purtscher视网膜病变类合计','缺血性视网膜病变类合计', '息肉样脉络膜血管病变PCV类合计','色素膜炎类合计', '后发障类合计',
                     '斜视类合计', '人工晶体眼类合计','脉络膜骨瘤类合计', '特发性脉络膜新生血管类合计', '视网膜前膜类合计',
                      '视神经病变类合计','病理性近视类合计', '星状玻璃体变性类合计', '晶体脱位类合计',
                     '高血压性视网膜病变类合计', '巩膜炎类合计', '闪辉性玻璃体液化类合计', '眼底出血类合计', '虹膜炎类合计',
                     '无晶状体眼类合计', '视网膜大动脉瘤类合计', '眼底血管样条纹类合计', '黑色素细胞瘤类合计','后发障类合计']

In [ ]:
# df_wdb_last_05.shape[0],df_wdb_last_21.shape[0],

In [ ]:
wdb_table['未打标眼底图数量'] = [df_wdb_last_tw.shape[0],df_wdb_last_bnz.shape[0],df_wdb_last_qgy.shape[0],
                         df_wdb_last_qgbz.shape[0],df_wdb_last_hbbb.shape[0],df_wdb_last_xgzs.shape[0],
                         df_wdb_last_xgy.shape[0],df_wdb_last_ys.shape[0],df_wdb_last_dmyh.shape[0],
                         df_wdb_last_swtl.shape[0],df_wdb_last_ssspgb.shape[0],df_wdb_last_mlmws.shape[0],
                         df_wdb_last_mlmwsmxgl.shape[0],df_wdb_last_mlmqs.shape[0],df_wdb_last_hssl.shape[0],
                         df_wdb_last_ssz.shape[0],df_wdb_last_mlmbb.shape[0],df_wdb_last_blthz.shape[0],
                         df_wdb_last_blthzthtl.shape[0],df_wdb_last_spsz.shape[0],df_wdb_last_bltcx.shape[0],
                         df_wdb_last_dsb.shape[0],df_wdb_last_spszqs.shape[0],df_wdb_last_spfyyc.shape[0],
                         df_wdb_last_ssjy.shape[0],df_wdb_last_01.shape[0],df_wdb_last_02.shape[0],df_wdb_last_03.shape[0],
                         df_wdb_last_04.shape[0],df_wdb_last_06.shape[0],df_wdb_last_07.shape[0],
                         df_wdb_last_08.shape[0],df_wdb_last_09.shape[0],df_wdb_last_10.shape[0],df_wdb_last_11.shape[0],
                         df_wdb_last_12.shape[0],df_wdb_last_13.shape[0],df_wdb_last_14.shape[0],df_wdb_last_15.shape[0],
                         df_wdb_last_16.shape[0],df_wdb_last_17.shape[0],df_wdb_last_18.shape[0],df_wdb_last_19.shape[0],
                         df_wdb_last_20.shape[0],df_wdb_last_22.shape[0],df_wdb_last_23.shape[0],
                         df_wdb_last_24.shape[0],df_wdb_last_25.shape[0],df_wdb_last_26.shape[0],df_wdb_last_27.shape[0],
                         df_wdb_last_28.shape[0],df_wdb_last_29.shape[0],df_wdb_last_30.shape[0],df_wdb_last_31.shape[0],
                         df_wdb_last_32.shape[0],df_wdb_last_33.shape[0],df_wdb_last_34.shape[0],df_wdb_last_35.shape[0]]

In [ ]:
wdb_table   # 未打标疾病标签统计

In [ ]:
ydb_table  # # 已打标疾病标签统计

In [ ]:
ydb_table[ydb_table['疾病子类'].str.contains('后发障')]

In [ ]:
# 最终的结果统计表
statistic_table = pd.merge(ydb_table,wdb_table,how='left',on='疾病子类')
statistic_table

In [ ]:
statistic_table['可打标眼底图数量'] = statistic_table.loc[:,['已打标眼底图数量','未打标眼底图数量']].apply(lambda x: x.sum(),axis=1)
# 将数据类型转为int型
statistic_table[['已打标眼底图数量','可打标眼底图数量']] = statistic_table[['已打标眼底图数量','可打标眼底图数量']].astype(int)  

In [ ]:
statistic_table.drop_duplicates(subset='疾病子类',keep='first',inplace=True)
statistic_table.reset_index(drop=True,inplace=True)
statistic_table

In [ ]:
statistic_table['已打标眼底图数量/总眼底图的占比'] = statistic_table['已打标眼底图数量']/df_last_yd.shape[0]
statistic_table['已打标眼底图数量/可打标眼底图的占比'] = statistic_table['已打标眼底图数量']/statistic_table['可打标眼底图数量']
# 设置显示格式为百分比
statistic_table['已打标眼底图数量/总眼底图的占比'] = statistic_table['已打标眼底图数量/总眼底图的占比'].map(lambda x:format(x,'.2%'))
statistic_table['已打标眼底图数量/可打标眼底图的占比'] = statistic_table['已打标眼底图数量/可打标眼底图的占比'].map(lambda x:format(x,'.2%'))

### 导出最终的结果统计表

In [ ]:
246-59+3

In [ ]:
out_put_path = r'/Users/edz/Documents/yandi_data/AI_data/zn_dc_ai总表/总表_Last'
out_put_file = r'/眼底图像的疾病打标情况统计表_last190.xlsx'
writer = pd.ExcelWriter(out_put_path + out_put_file)
statistic_table.to_excel(writer,sheet_name='眼底图像的疾病打标情况统计表',index=None)
writer.save()

In [ ]:
da = ydb_table[ydb_table['疾病子类'].str.contains('类合计')]

In [ ]:
zi_df = ydb_table.append(da)

In [ ]:
zi_df.drop_duplicates(subset='疾病子类',keep=False,inplace=True)

In [ ]:
zi_df['已打标眼底图数量'] = zi_df['已打标眼底图数量'].astype(int)
zi_df.sort_values(by='已打标眼底图数量',inplace=True,ascending=False)
zi_df.reset_index(drop=True,inplace=True)
zi_df

In [ ]:
zi_df_top30 =zi_df.iloc[:30,:] 
zi_df_top30

In [ ]:
output_path = r'/Users/edz/Documents/yandi_data/AI_data/标签汇总表'
output_file = r'/疾病190子类条形图_top30.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
zi_df.to_excel(writer,sheet_name='疾病190种子类条形图_全',index=None)
zi_df_top30.to_excel(writer,sheet_name='疾病190子类条形图Top30',index=None)
writer.save()

### 绘制疾病标签占比饼图

In [ ]:
# 选取含”类合计“的项
da_data = ydb_table[ydb_table['疾病子类'].str.contains('类合计')]
# 合并“糖、青、白”
da_data_add = ydb_table.loc[0:2,:]
da_data = da_data.append(da_data_add, ignore_index=True) 
da_data

In [ ]:
da_data['已打标眼底图数量'] = da_data['已打标眼底图数量'].astype(int)
da_data = da_data.sort_values(by='已打标眼底图数量', ascending=False)

In [ ]:
# pandas.Series.str.replace进行局部名称替换
da_data['疾病子类'] = da_data['疾病子类'].str.replace('合计','',regex=True)

In [ ]:
da_data.drop_duplicates(subset="疾病子类",keep='first',inplace=True)
da_data.reset_index(drop=True,inplace=True)

In [ ]:
da_data

In [ ]:
da_data_top30 = da_data.iloc[:30,:] 
da_data_top30

In [ ]:
output_path = r'/Users/edz/Documents/yandi_data/AI_data/标签汇总表'
output_file = r'/59大类眼底疾病条形图_top30.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
da_data.to_excel(writer,sheet_name='59大类眼底疾病条形图_全',index=None)
da_data_top30.to_excel(writer,sheet_name='59大类眼底疾病条形图Top30',index=None)
writer.save()

### 绘图

In [ ]:
from pylab import *
mpl.rcParams['font.sans-serif'] = ['Songti SC']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as sci

In [ ]:
def auto_text(rects):
    for rect in rects:
        ax.text(rect.get_x(), rect.get_height(), rect.get_height(), ha='left', va='bottom')

In [ ]:
# 设置图片大小
plt.figure(figsize=(20, 18))
# pal = sns.color_palette("Greens_d",len(da_data_top30['疾病子类'])
# 绘制水平图
sns.set(context='notebook',style='darkgrid',palette='deep',font='Songti SC',font_scale=1.2)
ax = sns.barplot( da_data_top30['疾病子类'],da_data_top30['已打标眼底图数量'],palette="Set1")
plt.xlabel('已打标眼底图数量',fontsize=15)
plt.ylabel('疾病子类',fontsize=15)

for column,row in da_data_top30.iterrows():
#     print(row)
    ax.text(row.疾病子类,row.已打标眼底图数量,'row.已打标眼底图数量')

# #为横向水平的柱图右侧添加数据标签。
# for rect in rect1 :
#     w = rect.get_hight()
#     ax.text(rect.get_x(), rect.get_height(), rect.get_height(), ha='left', va='bottom')

plt.show()

In [ ]:
# ax[0].get_height()
 for rect in rect1 :
    w = rect.get_hight()
    ax.text(rect.get_x(), rect.get_height(), rect.get_height(), ha='left', va='bottom')

In [ ]:
da_data_top30

In [ ]:
for column,row in da_data_top30.iterrows():
    print(row.已打标眼底图数量)
# ax.text(row.疾病子类,row.已打标眼底图数量)

In [ ]:
 for index,row in da_data_top30['已打标眼底图数量'].iterrows():
        print(index,row)

In [ ]:
city_name = da_data_top30['疾病子类'].to_list
city_name

In [ ]:
# 添加占比数据列
a = statistic_table
a['已打标眼底图数量/总眼底图的占比'] = a['已打标眼底图数量']/df_last_yd.shape[0]
a['已打标眼底图数量/可打标眼底图的占比'] = a['已打标眼底图数量']/a['可打标眼底图数量']
# 设置显示格式为百分比
a['已打标眼底图数量/总眼底图的占比'] = a['已打标眼底图数量/总眼底图的占比'].map(lambda x:format(x,'.2%'))
a['已打标眼底图数量/可打标眼底图的占比'] = a['已打标眼底图数量/可打标眼底图的占比'].map(lambda x:format(x,'.2%'))

In [ ]:
df_last_yd.shape[0]

In [ ]:
plt.pie(x, label, explode,shadow=True,pctdistance=0.6,labeldistance=1.1，startangle=90) # explode表示部分扇形突出，x,label,explode均为数组形式数据

In [ ]:
b = a[['疾病子类','已打标眼底图数量/可打标眼底图的占比']]

In [ ]:
c = b.drop([x for x in range(3,11)],axis=0)

In [ ]:
c = c.drop([x for x in range(34,42)],axis=0)
c 

In [ ]:
c

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['STHeiti'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
matplotlib.matplotlib_fname()  # 确认字体的安装目录

In [ ]:
from matplotlib import font_manager
x = sorted([f.name for f in font_manager.fontManager.ttflist])
for i in x:
    print(i)

In [ ]:
plt.rcParams['font.sans-serif']=['STHeiti'] #用来正常显示中文标签

In [ ]:
d = a[['疾病子类','已打标眼底图数量/总眼底图的占比']]

In [ ]:
d = d.drop([11,15,33,42],axis=0)
d

In [ ]:
# Hiragino Sans GB
# Songti SC
# Heiti TC
# PingFang HK
# Songti SC
import matplotlib

font = {'family' : 'Songti SC',
        'weight' : 'bold',
        'size'   : 10}
matplotlib.rc("font", **font)

In [ ]:
plt.rc(font)
fig = plt.figure()
plt.pie(d['已打标眼底图数量/总眼底图的占比'],labels=d['疾病子类'],autopct='%1.2f%%') #画饼图（数据，数据对应的标签，百分数保留两位小数点）
plt.title("已打标眼底图数量/总眼底图的占比分布")
plt.show()

### 导出没有32分类标签的原始数据，并拆分为6份

In [ ]:
# 导出没有32分类标签的原始数据
df_zong.shape

In [ ]:
df_zong.columns

In [ ]:
no_label = df_zong[['图像地址','arithmeticTag']]

In [ ]:
no_label = no_label[~no_label['图像地址'].str.contains('empty|invalid')]

In [ ]:
no_label = no_label[no_label['arithmeticTag'].isnull()]

In [ ]:
no_label[no_label['图像地址'].duplicated()==True]['图像地址'].value_counts()

In [ ]:
no_label.drop_duplicates(subset='图像地址',keep='first',inplace=True)

In [ ]:
no_label['bucket'] = 'zhizhen-tms'

In [ ]:
no_label.tail(2)

In [ ]:
no_label.shape[0]

In [ ]:
6*52400-366554

In [ ]:
no_label.drop(columns=['arithmeticTag'],inplace=True)

In [ ]:
no_label.rename(columns={'图像地址':'objectkey'},inplace=True)

In [ ]:
no_label.reset_index(drop=True,inplace=True)

In [ ]:
# 拆分数据表
start_x = 0
end_x = 52400
temp_size = 52400
for i in range(7):
    print(start_x,end_x)
    df_x = no_label.loc[start_x:end_x,:]
    print(df_x.shape[0])
    output_path = r'/Users/edz/Documents/yandi_data/AI_data/32分类标签/原始数据'
    output_path_file = r'/noclassific_part_' + str(i) + '.txt' 
    df_x.to_csv(output_path+output_path_file,sep='\t',index=None)
#     writer = pd.ExcelWriter(output_path+output_path_file)
#     df_x.to_excel(writer,sheet_name=str(start_x)+"-"+str(end_x),index=None)
#     writer.save()
    
    start_x += temp_size
    end_x += temp_size